<a href="https://colab.research.google.com/github/anushka-0907/HealthCare-Diagnosis-Assistant/blob/main/Healthcare_Diagnosis_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Healthcare Diagnosis Assistant 🏥 using BERT, NER & Medical Ontology
# --------------------------------------------------------------



In [1]:
from google.colab import files

In [2]:
uploaded = files.upload()

Saving Disease precaution.csv to Disease precaution.csv
Saving DiseaseAndSymptoms.csv to DiseaseAndSymptoms.csv


In [3]:
!pip install transformers torch scikit-learn pandas numpy



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import torch
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')



In [6]:

# Load datasets
symptom_df = pd.read_csv("DiseaseAndSymptoms.csv")
precaution_df = pd.read_csv("Disease precaution.csv")
symptom_df.fillna("", inplace=True)
precaution_df.fillna("", inplace=True)

display(symptom_df.head())
display(precaution_df.head())



,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,,,,,,,,,,,,,
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,,,,,,,,,,,,,,
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,,,,,,,,,,,,,,
3,Fungal infection,itching,skin_rash,dischromic _patches,,,,,,,,,,,,,,
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,,,,,,,,,,,,,,


,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up
1,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out
2,Allergy,apply calamine,cover area with bandage,,use ice to compress itching
3,Hypothyroidism,reduce stress,exercise,eat healthy,get proper sleep
4,Psoriasis,wash hands with warm soapy water,stop bleeding using pressure,consult doctor,salt baths


In [7]:

# Combine all symptom columns into one list per row
symptom_cols = [col for col in symptom_df.columns if col.startswith("Symptom")]
symptom_df["All_Symptoms"] = symptom_df[symptom_cols].values.tolist()
symptom_df["All_Symptoms"] = symptom_df["All_Symptoms"].apply(lambda x: [s.strip().lower().replace(" ", "_") for s in x if s])

# Encode symptoms
mlb = MultiLabelBinarizer()
X = mlb.fit_transform(symptom_df["All_Symptoms"])
y = symptom_df["Disease"]



In [8]:
#Train a Model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)
print("Model trained on symptom data.")



Model trained on symptom data.


In [9]:
#Use BERT for Named Entity Recognition
ner = pipeline("ner", grouped_entities=True)

def extract_symptoms(text):
    entities = ner(text)
    return [e["word"].strip().lower().replace(" ", "_") for e in entities if e["entity_group"] == "MISC" or e["entity_group"] == "ORG" or e["entity_group"] == "PER"]



No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Device set to use cuda:0


In [11]:
#User Input for Diagnosis
user_input = "I have stomach pain and have been vomiting since yesterday."
print(f"User input: {user_input}")

# Use NER to extract entities
symptoms_extracted = extract_symptoms(user_input)
print(f"Extracted symptoms: {symptoms_extracted}")

# Ontology Mapping - simple cleaning
ontology_symptoms = [s for s in symptoms_extracted if s in mlb.classes_]
print(f"Mapped symptoms: {ontology_symptoms}")

# Vectorize input
X_input = mlb.transform([ontology_symptoms])

# Predict disease
predicted_disease = model.predict(X_input)[0]
print(f"Predicted disease: {predicted_disease}")



User input: I have stomach pain and have been vomiting since yesterday.
Extracted symptoms: []
Mapped symptoms: []
Predicted disease: Dimorphic hemmorhoids(piles)


In [12]:
#Suggest Precautions
precautions = precaution_df[precaution_df["Disease"] == predicted_disease].values.tolist()
if precautions:
    print(f"Suggested Precautions:")
    for p in precautions[0][1:]:
        if pd.notna(p):
            print("-", p)
else:
    print("No precautions found for this disease.")

Suggested Precautions:
- avoid fatty spicy food
- consume witch hazel
- warm bath with epsom salt
- consume alovera juice
